In [1]:
import os
import xarray as xr
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from cmocean import cm
import cartopy.crs as ccrs
img = plt.imread('/path/to/BlueMarble_TOPO_010_JAN.png')

from matplotlib import animation as anim
from IPython.display import HTML

# If the simulation you want to create is going to be large, you need to set this following parameter
# (I think it defaults to somewhere around 20mb) We set it to 200mb here.
matplotlib.rcParams['animation.embed_limit'] = 200.

We use Optimum Interpolation Sea Surface Temperature (OISST) v2.1 data provided by NOAA ([link](https://www.ncdc.noaa.gov/oisst/data-access)) for this example.  

The background image is taken from the NASA visible earth collection Blue Marble ([link](https://visibleearth.nasa.gov/collection/1484/blue-marble)) 

First, we need to construct a list with the file names that we want to access. In this case, we want to plot the first two months of 2020 but it's easy to extend that to the full year by changing the range of the `for` loop to `range(1, 13)`.

In [2]:
file_base = "https://www.ncei.noaa.gov/thredds/dodsC/OisstBase/NetCDF/V2.1/AVHRR/2020"
file_list = []

len_mon = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
for m in range(1, 3):
    l_mon = len_mon[m - 1]
    m2 = "%02d" % m
    file_ext = file_base + m2 + "/oisst-avhrr-v02r01.2020"
    for d in range(1, l_mon + 1):
        d2 = "%02d" % d
        file_list.append(file_ext + str(m2) + str(d2) + ".nc")

We use `xarray` to open all OPENDAP links in `file_list` to have them in one dataset.

In [3]:
ds = xr.open_mfdataset(file_list)

We need to define a new class for the projection to change the `threshold` value in order to draw smooth lines with `plot`

In [4]:
class LowerThresholdNearsidePerspective(ccrs.NearsidePerspective):

    @property
    def threshold(self):
        return 1000

### Now the animation

In [8]:
plt.ioff() # we don"t want to see the figures after executing this cell
# First, we define some levels for the colors we want to use in order to create a discrete (and not continuous)
# colormap
levs = np.arange(-2, 32, 1)
norm = colors.BoundaryNorm(boundaries=levs, ncolors=256)
# Now we define how long the animation should be (for testing this could be put to low numbers)
len_anim = len(ds.time)

# Now define a range of central longitudes (during the animation, the central longitude will vary over this range)
cen_lon = np.linspace(40.0, 60.0, len_anim)

# Then we set up the figure
fig = plt.figure(figsize=(8,4), dpi=50) # set `dpi` higher for higher quality saved animations in the end
plt.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.9)

# And create a function for `FuncAnimation` that plots the figures (frames) of the animation
def animate(i):
    plt.clf()
    # We set up the projection to be used
    m1 = fig.add_subplot(1, 1, 1, projection=LowerThresholdNearsidePerspective(central_longitude=cen_lon[i],
                                                                           central_latitude=-45,
                                                                           satellite_height=25000000,
                                                                           globe=None))
    # Now we display the background image
    # You can lower the value of `regrid_shape` to make the plotting faster, however this has negative effects
    # on the quality of the displayed background image
    m1.imshow(img, origin='upper', transform=ccrs.PlateCarree(),
              interpolation='spline36', regrid_shape=400, zorder=1)
    # And plot the SST data
    map1 = m1.pcolormesh(ds.lon, ds.lat, ds.sst.isel(time=i).squeeze(),
                  cmap=cm.thermal, norm=norm, transform=ccrs.PlateCarree(), zorder=2)
    # We add a colorbar and its label
    c = plt.colorbar(map1, pad=0.04, shrink=0.4, orientation='horizontal')
    c.ax.set_ylabel('$^{\circ}C$');
    # and a title
    m1.set_title("OISST at " + str(ds.time.isel(time=i).values)[0:10]);
    # Now we plot a line into the map (this could be the region you are interested in)
    line1 = m1.plot([30, 70, 70, 30, 30],
                    [-35, -35, -50, -50, -35],
                    linewidth=2, color='gold', transform=ccrs.PlateCarree(), zorder=3)
    # set the latitude limits of the plot. we do that in projection coordinates (m)
    # because there are some problems when using latitude
    m1.set_ylim(-3000000, 3200000)
    
# And finally, we create the animation
sst_anim = anim.FuncAnimation(fig, animate, frames=len_anim, interval=100)

In [9]:
# And now we display the animation inside the notebook with `HTML`
# Depending on the number of frames you want to plot, the construction of the animation might take a while!
HTML(sst_anim.to_jshtml())

In [10]:
# To save the animation
writer = anim.FFMpegWriter(bitrate=-1, codec="libx264", extra_args=['-pix_fmt', 'yuv420p'])
sst_anim.save("test_animation.mp4", writer=writer, dpi=50) # set `dpi` higher for better quality